In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

import os
import copy
import time
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
data_dir = 'dataset/food11'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'test')

BATCH_SIZE = 32
NUM_CLASSES = 11

In [ ]:
IMG_SIZE_B0 = 224

data_transforms_b0 = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(IMG_SIZE_B0),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(IMG_SIZE_B0), 
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms_b0[x])
    for x in ['train', 'test']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    for x in ['train', 'test']
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

In [ ]:
model_b0 = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

for param in model_b0.parameters():
    param.requires_grad = False

num_features = model_b0.classifier[1].in_features

model_b0.classifier[1] = nn.Linear(num_features, NUM_CLASSES)

model_b0 = model_b0.to(device)

loss = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_b0.classifier.parameters(), lr=0.001)

In [12]:
def train_model(model, crit, optimizer, num_epochs=10):
    since = time.time()
    best_model_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
                description = f'Train epoch {epoch+1}/{num_epochs}'
            else:
                model.eval()
                description = f'Test epoch {epoch+1}/{num_epochs}'
            
            total_loss = 0.0
            corrects = 0.0

            train_tqdm = tqdm(
                dataloaders[phase],
                desc=description,
                leave=True
            )
            for batch_idx, (images, labels) in enumerate(train_tqdm):
                images = images.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    out = model(images)
                    _, preds = torch.max(out, 1)
                    loss = crit(out, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                total_loss += loss.item() * images.size(0)
                corrects += torch.sum(preds == labels.data) 

                current_loss = total_loss / (batch_idx + 1) / dataloaders[phase].batch_size
                current_acc = corrects.double() / (batch_idx + 1) / dataloaders[phase].batch_size

                train_tqdm.set_postfix(
                    Loss=f'{current_loss:.4f}', 
                    Acc=f'{current_acc:.4f}'
                )
            epoch_acc = corrects / (batch_idx + 1) / dataloaders[phase].batch_size

            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_weights = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'best_model_food11_b0.pth')
    
    print()
    time_elapsed = time.time() - since
    print(f'Обучение завершено за {time_elapsed // 60:.0f}м {time_elapsed % 60:.0f}с')
    print(f'Лучшая точность на валидации: {best_acc:.4f}')

    model.load_state_dict(best_model_weights)
    return model

In [13]:
model_ft_b0 = train_model(model_b0, loss, optimizer, num_epochs=5)

Test epoch 5/5: 100%|██████████| 35/35 [00:05<00:00,  6.91it/s, Acc=0.7839, Loss=0.6519]


Обучение завершено за 2м 26с
Лучшая точность на валидации: 0.7839


In [17]:
for param in model_ft_b0.features[-3:].parameters():
    param.requires_grad = True

optimizer_ft_b0 = optim.Adam(model_ft_b0.parameters(), lr=1e-5)

In [18]:
model_fine_tuned_b0 = train_model(
    model_ft_b0,
    loss, 
    optimizer_ft_b0,
    num_epochs=10
)

torch.save(model_fine_tuned_b0.state_dict(), 'best_model_food11_b0_FINETUNED.pth')

Test epoch 10/10: 100%|██████████| 35/35 [00:05<00:00,  6.53it/s, Acc=0.8321, Loss=0.4549]


Обучение завершено за 5м 50с
Лучшая точность на валидации: 0.8321


In [14]:
from PIL import Image
import numpy as np

image_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE_B0),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

image_path = 'src/pizza.jpg'
image = Image.open(image_path).convert('RGB')
image_tensor = image_transform(image)

input_batch = image_tensor.unsqueeze(0)

input_batch = input_batch.to(device)

model_b0.eval()

with torch.no_grad():
    out = model_b0(input_batch)

    probabilities = nn.functional.softmax(out, dim=1)

    top_p, top_class_index = probabilities.topk(1, dim=1)

    predicted_class_name = class_names[top_class_index.item()]
    predicted_probability = top_p.item()

print(f"Предсказанный класс: {predicted_class_name}")
print(f"Вероятность: {predicted_probability:.4f}")

Предсказанный класс: pizza
Вероятность: 0.9460


In [ ]:
IMG_SIZE_B1 = 240

data_transforms_b1 = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(IMG_SIZE_B1),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(IMG_SIZE_B1), 
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms_b1[x])
    for x in ['train', 'test']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    for x in ['train', 'test']
}

In [10]:
model_b1 = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.IMAGENET1K_V1)

for param in model_b1.parameters():
    param.requires_grad = False

num_features = model_b1.classifier[1].in_features

model_b1.classifier[1] = nn.Linear(num_features, NUM_CLASSES)

model_b1 = model_b1.to(device)

loss = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_b1.classifier.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-bac287d4.pth" to C:\Users\masha/.cache\torch\hub\checkpoints\efficientnet_b1_rwightman-bac287d4.pth


100%|██████████| 30.1M/30.1M [00:01<00:00, 24.4MB/s]


In [15]:
model_ft_b1 = train_model(model_b1, loss, optimizer, num_epochs=5)

Test epoch 5/5: 100%|██████████| 35/35 [00:05<00:00,  6.32it/s, Acc=0.7839, Loss=0.6609]


Обучение завершено за 2м 51с
Лучшая точность на валидации: 0.7839
